In [ ]:
import csv

Решил исследовать dataset anime. Источник: https://github.com/caserec/Datasets-for-Recommender-Systems/tree/master/Processed%20Datasets/Anime

Первоначально столкнулся с проблемой обработки жанров, как важного элемента рекоммендаций. Данные были представлены в формате:

| anime_ids | name                          | genre                                             | type  | episodes | rating | members |
|-----------|-------------------------------|---------------------------------------------------|-------|----------|--------|---------|
| 1723      | Kimi no Na wa.                | Drama, Romance, School, Supernatural            | Movie | 1        | 9.37   | 200630  |
| 82        | Fullmetal Alchemist: Brotherhood | Action, Adventure, Drama, Fantasy, Magic, Military, Shounen | TV    | 64       | 9.26   | 793665  |


В таком виде алгоритмы будут строить эмбеддинги, которые сложно будет сравнивать друг с другом(много уникальных сочетаний жанров). Поэтому оценил кол-во фильмов, которое останется при отборе самых популярных жанров:

In [ ]:
def count_filtered_titles(dat_file, target_genres):
    total_titles = 0
    matching_titles = 0

    with open(dat_file, 'r', encoding='utf-8') as file:
        next(file)
        for line in file:
            total_titles += 1
            columns = line.strip().split('\t')
            if len(columns) < 3:
                continue
            genres = columns[2].split(',')
            genres = {g.strip() for g in genres}

            if any(genre in genres for genre in target_genres):
                matching_titles += 1

    return matching_titles


dat_file = 'anime_info.dat'
target_genres = {
    'Comedy', 'Action', 'Sci-Fi', 'Adventure', 'Fantasy', 'Drama', 'Shounen',
    'Romance','Shoujo', 'Seinen'
}

matching_titles = count_filtered_titles(dat_file, target_genres)
print(f"Number of titles with at least one of the specified genres: {matching_titles}")


Небольшая потеря в сравнении с 7390 изначальными произведениями

Сами жанры переформатировал по принципу One Hot encoder, а так же оставил только нужные колонки:

In [ ]:
def process_anime_dataset(input_file, output_file):
    target_genres = {
        'Comedy', 'Action', 'Sci-Fi', 'Adventure', 'Fantasy',
        'Drama', 'Shounen', 'Romance', 'Shoujo', 'Seinen'
    }

    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', encoding='utf-8', newline='') as outfile:

        reader = csv.DictReader(infile, delimiter='\t')
        fieldnames = ['anime_ids', 'type', 'episodes', 'rating', 'members'] + list(target_genres)
        writer = csv.DictWriter(outfile, fieldnames=fieldnames, delimiter='\t')
        writer.writeheader()

        for row in reader:
            genres = set(g.strip() for g in row['genre'].split(','))

            if not any(g in genres for g in target_genres):
                continue

            output_row = {
                'anime_ids': row['anime_ids'],
                'type': row['type'],
                'episodes': row['episodes'],
                'rating': row['rating'],
                'members': row['members']
            }

            for genre in target_genres:
                output_row[genre] = 1 if genre in genres else 0

            writer.writerow(output_row)


input_file = 'anime_info.dat'
output_file = 'filtered_anime.dat'
process_anime_dataset(input_file, output_file)


Данные приняли вид:

| anime_ids | type  | episodes | rating | members | Drama | Adventure | Shoujo | Action | Seinen | Sci-Fi | Fantasy | Comedy | Shounen | Romance |
|-----------|-------|----------|--------|---------|-------|-----------|--------|--------|--------|--------|---------|--------|---------|---------|
| 1723      | Movie | 1        | 9.37   | 200630  | 1     | 0         | 0      | 0      | 0      | 0      | 0       | 0      | 0       | 1       |
| 82        | TV    | 64       | 9.26   | 793665  | 1     | 1         | 0      | 1      | 0      | 0      | 1       | 0      | 1       | 0       |
| 296       | TV    | 51       | 9.25   | 114262  | 0     | 0         | 0      | 1      | 0      | 1      | 0       | 1      | 1       | 0       |


В связи с удалением некоторых фильмов нужно было зачистить связанные отзывы

In [ ]:
import csv

def extract_anime_ids(filtered_anime_file):
    anime_ids = set()
    with open(filtered_anime_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter='\t')
        for row in reader:
            anime_ids.add(row['anime_ids'])
    return anime_ids

def filter_ratings(rating_file, output_file, anime_ids):
    with open(rating_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', encoding='utf-8', newline='') as outfile:

        reader = csv.DictReader(infile, delimiter='\t')
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames, delimiter='\t')
        writer.writeheader()

        for row in reader:
            if row['Anime_ID'] in anime_ids:
                writer.writerow(row)


filtered_anime_file = 'filtered_anime.dat'
rating_file = 'anime_ratings.dat'
output_file = 'filtered_ratings.dat'

anime_ids = extract_anime_ids(filtered_anime_file)
filter_ratings(rating_file, output_file, anime_ids)

В итоге осталось 400К из изначальных 500К, более чем достаточно для наших целей

EDA и визуализации

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ratings_df = pd.read_csv('filtered_ratings.dat', sep='\t')

anime_df = pd.read_csv('filtered_anime.dat', sep='\t')


Рейтинг

In [ ]:
mean_rating = ratings_df['Feedback'].mean()
median_rating = ratings_df['Feedback'].median()

print(f"Средний рейтинг: {mean_rating:.2f}")
print(f"Медианный рейтинг: {median_rating:.2f}")

reviews_per_anime = ratings_df.groupby('Anime_ID').size()
mean_reviews_per_anime = reviews_per_anime.mean()
median_reviews_per_anime = reviews_per_anime.median()

print(f"Среднее количество отзывов на фильм: {mean_reviews_per_anime:.2f}")
print(f"Медианное количество отзывов на фильм: {median_reviews_per_anime:.2f}")

reviews_per_user = ratings_df.groupby('User_ID').size()
mean_reviews_per_user = reviews_per_user.mean()
median_reviews_per_user = reviews_per_user.median()

print(f"Среднее количество отзывов на пользователя: {mean_reviews_per_user:.2f}")
print(f"Медианное количество отзывов на пользователя: {median_reviews_per_user:.2f}")


In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Feedback', data=ratings_df)
plt.title('Распределение рейтингов')
plt.xlabel('Рейтинг')
plt.ylabel('Количество')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(reviews_per_anime, bins=20)
plt.title('Распределение количества отзывов на фильм')
plt.xlabel('Количество отзывов')
plt.ylabel('Количество фильмов')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(reviews_per_user, bins=20)
plt.title('Распределение количества отзывов на пользователя')
plt.xlabel('Количество отзывов')
plt.ylabel('Количество пользователей')
plt.show()


Аниме

In [ ]:
mean_episodes = anime_df['episodes'].mean()
median_episodes = anime_df['episodes'].median()

print(f"Среднее количество эпизодов: {mean_episodes:.2f}")
print(f"Медианное количество эпизодов: {median_episodes:.2f}")

type_counts = anime_df['type'].value_counts()
print("\nКоличество каждого типа:")
print(type_counts)

mean_rating_anime = anime_df['rating'].mean()
median_rating_anime = anime_df['rating'].median()

print(f"\nСредний рейтинг аниме: {mean_rating_anime:.2f}")
print(f"Медианный рейтинг аниме: {median_rating_anime:.2f}")

genre_columns = ['Drama', 'Adventure', 'Shoujo', 'Action', 'Seinen', 'Sci-Fi', 'Fantasy', 'Comedy', 'Shounen', 'Romance']
genre_counts = anime_df[genre_columns].sum().sort_values(ascending=False)
print("\nКоличество каждого жанра:")
print(genre_counts)


In [ ]:
bins = [0, 7, 12, 24, 50, 100, float('inf')]
labels = ['1-7', '8-12', '13-24', '25-50', '51-100', '100+']
anime_df['episode_bins'] = pd.cut(anime_df['episodes'], bins=bins, labels=labels, right=False)

plt.figure(figsize=(10, 6))
sns.countplot(x='episode_bins', data=anime_df)
plt.title('Распределение количества эпизодов')
plt.xlabel('Количество эпизодов')
plt.ylabel('Количество аниме')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(anime_df['rating'], bins=20)
plt.title('Распределение рейтинга аниме')
plt.xlabel('Рейтинг')
plt.ylabel('Количество аниме')
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x=genre_counts.index, y=genre_counts.values)
plt.title('Количество каждого жанра')
plt.xlabel('Жанр')
plt.ylabel('Количество')
plt.xticks(rotation=45)
plt.show()


Качество данных

In [ ]:
print("Пропуски в данных о рейтингах:")
print(ratings_df.isnull().sum())

print("\nПропуски в данных об аниме:")
print(anime_df.isnull().sum())


In [ ]:
duplicate_ratings = ratings_df.duplicated().sum()
print(f"\nКоличество дубликатов в данных о рейтингах: {duplicate_ratings}")

duplicate_anime = anime_df.duplicated(subset='anime_ids').sum()
print(f"Количество дубликатов в данных об аниме: {duplicate_anime}")


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=anime_df['episodes'])
plt.title('Выбросы в количестве эпизодов')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x=anime_df['rating'])
plt.title('Выбросы в рейтингах')
plt.show()
